In [2]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import pandas as pd
from datetime import datetime
import time
import os
import re
import sys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException

In [3]:
service = Service('/usr/lib/chromium-browser/chromedriver')
options = Options()
options.add_argument('--incognito')
options.add_argument('start-maximized')
driver = webdriver.Chrome(service=service, options=options)

In [4]:
df = pd.DataFrame(columns=['name','place','details'])
URL=f"https://www.haaretz.co.il/news/2023-10-12/ty-article-magazine/0000018b-1367-dcc2-a99b-17779a0a0000"
driver.get(URL)
time.sleep(5)
html = driver.page_source
soup = bs(html, features='html.parser')

In [5]:
frames = soup.findAll('div', class_='war-victims-card filterd')
i=0
for  frame in frames:
    i=i+1
    #print (f"row:{i}")

    if(frame.find('h3',class_='war-victims-card__name')):
        v_name=frame.find('h3',class_='war-victims-card__name').text
    else:
        v_name=' '

    if(frame.find('h3',class_='war-victims-card__details')):
        v_place=frame.find('h3',class_='war-victims-card__details').text
    else:
        v_place=' '

    if(frame.find('p',class_='war-victims-card__text')):
        v_text=frame.find('p',class_='war-victims-card__text').text.strip()
    else:
        v_text=' '

    #print (f"text:--{v_text}--")
    #print (f"name:--{v_name}--")
    #print (f"place:--{v_place}--")
    df.loc[len(df)] = [v_name,v_place,v_text]

print (f"number of victims: {i}")
#print (f"{frame}")


number of victims: 768


In [6]:
master_df=df.copy()
master_df


,name,place,details
0,"עומר בלוה, 22",הרצליה,"סמ""ר במילואים, נפל בפעילות מבצעית בגבול הצפון ..."
1,"חגי אפרת, 83",ראשון לציון,נרצח בפיגוע ירי במצרים
2,גרייסי קבררה,בארי,
3,מילה כהן,בארי,
4,אוהד כהן,בארי,
...,...,...,...
763,"ג'וועד אברהים אלקרעאן, 15",כוחלה,פגיעה ישירה של רקטה בצריף מגורים בכוחלה
764,"מאלק אברהים אלקרעאן, 14",כוחלה,פגיעה ישירה של רקטה בצריף מגורים בכוחלה
765,"יזן זכריה אבו ג'מאע, 5",ערערה,מת מפגיעת רקטה ליד ביתו בערערה
766,"אופיר ליבשטיין, 50",כפר עזה,"ראש המועצה האזורית שער הנגב,\n ..."


In [7]:
mil_ranks=[
    'רב סמל מתקדם',
    'רב סמל בכיר',
    'רב סמל ראשון',
    'סגן ניצב',
    'סמל ראשון',
    'טוראית',
    'טוראי',
    'רב״ט',
    'סמ״ר',
    'רס״ל',
    'רב סמל',
    'רס״ר',
    'רס״מ',
    'רס״ם',
    'רס״ב',
    'רנ״מ',
    'רנ״ם',
    'רנ״ג',
    'סג״מ',
    'סג״ם',
    'סגן',
    'סרן',
    'רס״נ',
    'רס״ן',
    'סא״ל',
    'אל״מ',
    'אל״ם',
    'תא״ל',
    'סמל'
]
police_ranks=['רב פקד','רב נגד','רב שוטר','סנ״צ','רפ״ק','פקד','מפקח',]

In [8]:
import numpy as np
df=master_df.copy()
df['age']=df.name.str.extract(',? (\d*) ?$')
df['name']=df.name.str.replace(',? (\d*) ?$','')

df['n_rank']=-1
for rnk in mil_ranks:
    #print(rnk)
    df['rnk']=df['name'].str.contains(rnk,regex=False)
    df['name']=df['name'].str.replace(rf"{rnk}",r"", regex=True)
    #df.n_rank = np.where(df.rnk, rnk,-1)
    df.n_rank[df['rnk']]=rnk

for rnk in police_ranks:
    df['rnk']=df['name'].str.contains(rnk,regex=False)
    df['name']=df['name'].str.replace(rf"{rnk}",r"", regex=True)
    #df.n_rank = np.where(df.rnk, rnk,-1)
    df.n_rank[df['rnk']]=rnk
     

df['mil']=df['name'].str.contains('מיל.\)',regex=True) 
df['name']=df['name'].str.replace(r'\(ב?מיל.?\)',r'',regex=True)
df['name']=df.name.str.strip()
#df['name']=df['name'].str.replace(r'(ב',r'',regex=False) 

/tmp/ipykernel_23525/922371872.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.n_rank[df['rnk']]=rnk
/tmp/ipykernel_23525/922371872.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'רב סמל מתקדם' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.n_rank[df['rnk']]=rnk
/tmp/ipykernel_23525/922371872.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.n_rank[df['rnk']]=rnk
/tmp/ipykernel_23525/922371872.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [9]:
df['emergencySquad']=df['details'].str.contains('כוננות',regex=False)
df['dance']=df['details'].str.contains('מסיב',regex=False)
df['battalion']=df.details.str.extract('גדוד (\d+)')
#df['details']=df.details.str.replace('גדוד (\d+)','')
df['brigade']=df.details.str.extract('חטיבה (\d+)')
#df['details']=df.details.str.replace('חטיבה (\d+)','')

brigades=['גולני','צנחנים','נח״ל','הצפונית','מרחבית יהודה','הדרומית','שומרון','מגלן','ימ״מ','דובדבן']
for j in brigades:
    df['rnk']=df['details'].str.contains(j,regex=False)
    #df['details']=df['details'].str.replace(rf"{j}",r"", regex=True)
    #df.n_rank = np.where(df.rnk, rnk,-1)
    df.brigade[df['rnk']]=j    


/tmp/ipykernel_23525/593391970.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.brigade[df['rnk']]=j
/tmp/ipykernel_23525/593391970.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.brigade[df['rnk']]=j
/tmp/ipykernel_23525/593391970.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.brigade[df['rnk']]=j
/tmp/ipykernel_23525/593391970.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [10]:
myDf=df.loc[:,["name","place","age","n_rank","emergencySquad","dance","battalion","brigade","details"]]

In [11]:
df.iloc[2,]

name              גרייסי קבררה
place                     בארי
details                       
age                        NaN
n_rank                      -1
rnk                      False
mil                      False
emergencySquad           False
dance                    False
battalion                  NaN
brigade                    NaN
Name: 2, dtype: object

In [12]:
timestr = time.strftime("%Y%m%d-%H%M%S")
myDf.to_csv(f"/home/gilad/Dropbox/SoI/victims_{timestr}.csv")
driver.close()

In [13]:
df['name'][443].strip()[0]
#print(ord(df['name'][64][2])) #443
#print(ord('"'))

'ע'

In [14]:
df

,name,place,details,age,n_rank,rnk,mil,emergencySquad,dance,battalion,brigade
0,"עומר בלוה, 22",הרצליה,"סמ""ר במילואים, נפל בפעילות מבצעית בגבול הצפון ...",22,-1,False,False,False,False,NaN,NaN
1,"חגי אפרת, 83",ראשון לציון,נרצח בפיגוע ירי במצרים,83,-1,False,False,False,False,NaN,NaN
2,גרייסי קבררה,בארי,,NaN,-1,False,False,False,False,NaN,NaN
3,מילה כהן,בארי,,NaN,-1,False,False,False,False,NaN,NaN
4,אוהד כהן,בארי,,NaN,-1,False,False,False,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
763,"ג'וועד אברהים אלקרעאן, 15",כוחלה,פגיעה ישירה של רקטה בצריף מגורים בכוחלה,15,-1,False,False,False,False,NaN,NaN
764,"מאלק אברהים אלקרעאן, 14",כוחלה,פגיעה ישירה של רקטה בצריף מגורים בכוחלה,14,-1,False,False,False,False,NaN,NaN
765,"יזן זכריה אבו ג'מאע, 5",ערערה,מת מפגיעת רקטה ליד ביתו בערערה,5,-1,False,False,False,False,NaN,NaN
766,"אופיר ליבשטיין, 50",כפר עזה,"ראש המועצה האזורית שער הנגב,\n ...",50,-1,False,False,False,False,NaN,NaN
